In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from kinematics import (
    trisphere_inverse_kinematics, rotation_x, rotation_y, rotation_z, 
    rotation_fixed_angle, get_plane_orientation
)

In [ ]:
jack1_p0 = np.array([1150, 100, 0])
jack2_p0 = np.array([0, 0, 0])
jack3_p0 = np.array([530, 970, 0])
jack_p0 = (jack1_p0, jack2_p0, jack3_p0)

jack1_p0_min = jack1_p0 - np.array([200, 0, 0])
jack1_p0_max = jack1_p0 + np.array([200, 0, 0])

jack2_p0_min = jack2_p0 - np.array([0, 200, 0])
jack2_p0_max = jack2_p0 + np.array([0, 200, 0])

jack3_p0_min = jack3_p0 - np.array([0, 200, 0])
jack3_p0_max = jack3_p0 + np.array([0, 200, 0])

jack_p0_min = (jack1_p0_min, jack2_p0_min, jack3_p0_min)
jack_p0_max = (jack1_p0_max, jack2_p0_max, jack3_p0_max)

c_p0 = np.array([300, 300, 300])
c_r0 = np.array([0, 0, 0])

In [ ]:
def visualize():
    """"""
    def visualize_plane(i, j, figsize):
        fig, ax = plt.subplots(figsize=figsize)

        # original vector
        ax.plot(c_p0[i], c_p0[j], 's', color='k', ms=10, label='Origin of control point')
        ax.arrow(c_p0[i], c_p0[j], 100*pointing0[i], 100*pointing0[j], 
                 head_width=20, head_length=40, color='k', alpha=0.7)
        
        # target vector
        ax.plot(c_dp[i] + c_p0[i], c_dp[j] + c_p0[j], 's', color='r', ms=10, label='Control point')
        ax.arrow(c_dp[i] + c_p0[i], c_dp[j] + c_p0[j], 100*pointing[i], 100*pointing[j], 
                 head_width=20, head_length=40, color='r', alpha=0.7)
        
        x = [p[i] for p in jack_p0]
        y = [p[j] for p in jack_p0]
        ax.plot(x, y, 'x', ms=19, color='r', label='Null position')
        for p_min, p_max in zip(jack_p0_min, jack_p0_max):
            ax.plot((p_min[i], p_max[i]), (p_min[j], p_max[j]), '-', lw=3, color='r', alpha=0.7)
        
        x = [p[i] for p in tp0]
        y = [p[j] for p in tp0]
        ax.plot(x, y, '+', ms=19, color='g', label='Transformed null position')
        for p_min, p_max in zip(tp0_min, tp0_max):
            ax.plot((p_min[i], p_max[i]), (p_min[j], p_max[j]), '-', lw=3, color='g', alpha=0.7)

        x = [p[i] + p0[i] for p, p0 in zip(jack_p, jack_p0)]
        y = [p[j] + p0[j] for p, p0 in zip(jack_p, jack_p0)]
        ax.plot(x, y, '.', ms=19, color='b', label='New position')

        ax.set_xlabel(labels[i], fontsize=20)
        ax.set_ylabel(labels[j], fontsize=20)
        ax.legend()
        ax.grid()
        ax.set_aspect('equal')
        plt.tight_layout()
   
    labels = ('x', 'y', 'z')

    visualize_plane(0, 1, (10, 10))
    visualize_plane(0, 2, (10, 5))
    visualize_plane(1, 2, (10, 5))


def check():
    def is_colinear(p1, p2, p3):
        a1 = p1[0]*(p2[1] - p3[1]) + p2[0]*(p3[1] - p1[1]) + p3[0]*(p1[1] - p2[1])
        a2 = p1[1]*(p2[2] - p3[2]) + p2[1]*(p3[2] - p1[2]) + p3[1]*(p1[2] - p2[2])
        a3 = p1[2]*(p2[0] - p3[0]) + p2[2]*(p3[0] - p1[0]) + p3[2]*(p1[0] - p2[0])
        ret = abs(a1) < 1e-6 and abs(a2) < 1e-6 and abs(a3) < 1e-6
        if ret:
            print("passed")
        else:
            print(a1, a2, a3)
            

    for p_min, p_max, p, p0 in zip(tp0_min, tp0_max, jack_p, jack_p0):
        is_colinear(p_min, p_max, p + p0)


c_dp = np.array([10., 20., -30.])
c_dr = np.radians(np.array([3, 0, 2]))  # gamma, beta, alpha

pointing0 = np.array([0., 1., 0.])
rt_matrix = rotation_fixed_angle(*c_dr)
pointing = (pointing0 * rt_matrix.T).getA()[0]

jack_p, tp0 = trisphere_inverse_kinematics(c_dp, c_dr, c_p0, c_r0, jack_p0)
jack_p_min, tp0_min = trisphere_inverse_kinematics(c_dp, c_dr, c_p0, c_r0, jack_p0_min)
jack_p_max, tp0_max = trisphere_inverse_kinematics(c_dp, c_dr, c_p0, c_r0, jack_p0_max)

for p in jack_p:
    print(p)

check() 
    
visualize()

print(get_plane_orientation(*jack_p))